# Transform mask into COSTO .json

In [229]:
import re
from os import listdir, PathLike
from os.path import join
from pipeline.image_handeling.data_utility import load_stack
from skimage.measure import find_contours, regionprops_table
from numpyencoder import NumpyEncoder
import json

def mask_to_json(mask_fold_path: PathLike, save_path: PathLike, channel:str)->None:
  """Function that converts .tif labeled masks from np.array stacks into COCO .json format files.
  Saves the file into the given location.
  Args:
      mask_fold_path (PathLike): Folder of the masks.
      save_path (PathLike): Path where the .json is going to be saved. Must include the filename and the ending ".json".
      channel (string): The channel that should be transformed into .json.
  Returns:
      None, saves the .json into the given location."""
  img_path = []
  for file in sorted(listdir(mask_fold_path)):
      if channel in file:
        img_path.append(join(mask_fold_path,file))
  im_num = re.findall('f\d+', str(img_path))
  max_number = max(int(x[1:]) for x in im_num)
  max_number

  img_stack = load_stack(img_paths=img_path, channels=channel, frame_range=range(max_number), return_2D=True)

  annotations_lst = []
  images_lst = []
  for frame, img in enumerate(img_stack):
    outline_region =regionprops_table(img, properties=('label','bbox','area'))
    frame_name = img_path[frame].rsplit('/',1)[1]
    labels = outline_region['label']
    bbox0 = outline_region['bbox-0']
    bbox1 = outline_region['bbox-1']
    bbox2 = outline_region['bbox-2']
    bbox3 = outline_region['bbox-3']
    area = outline_region['area']

    for i in range(labels.shape[0]):
        label = labels[i]
        segmentation_lst=[]
        regionmask=(img==label)
        coords = find_contours(regionmask)
        for y,x in coords[0]:
            segmentation_lst.append(x+0.5)
            segmentation_lst.append(y+0.5)
        
        annotations_lst.append({'id':label, 
                                'image_id':frame+1,
                                'category_id':1,
                                'segmentation':[segmentation_lst],
                                'area':area[i],
                                'bbox':[bbox0[i], bbox1[i], bbox2[i], bbox3[i]],
                                'iscrowd': 0,
                                'attributes': {'occluded': False}})

    images_lst.append({'id': frame+1,
              'width': img.shape[1],
              'height': img.shape[0],
              'file_name': frame_name,
              'license': 0,
              'flickr_url': '',
              'coco_url': '',
              'date_captured': 0})

  json_dict = {'licenses': [{'name': '', 'id': 0, 'url': ''}],
          'info': {'contributor': '',
            'date_created': '',
            'description': '',
            'url': '',
            'version': '',
            'year': ''},
          'categories': [{'id': 1, 'name': 'Cell', 'supercategory': ''}],
          'images': images_lst,
          'annotations':annotations_lst}

  with open(save_path, "w") as file:
      json.dump(json_dict, file, cls=NumpyEncoder)

In [230]:

mask_folder = '/home/Fabian/ImageData/TrackingTestFiles/NeutrophilTrackingTest/mfap4-mpx_isohypo_2h_WT-MaxIP_s1/Masks_Cellpose'
channel = 'GFP'
savefile = '/home/Fabian/ImageData/Seriestest.json'

mask_to_json(mask_fold_path=mask_folder, save_path=savefile, channel=channel)

[array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)]


# Reconstruct mask based on COSTO .json

In [231]:
import json
import numpy as np
from skimage.draw import polygon2mask
from os import PathLike

def mask_from_json(path:PathLike)->tuple[list[str],np.array]:
    """Recreates an np.array like stack of masks based on an COCO .json file.
    Return the np.array stack of masks and a list fo the Mask names.
    Args:
        path (PathLike): Path of the .json file to be loaded.
    Returns:
        tuple[list[str],np.array]: List of the image names, np.array containing the loaded masks"""
    file = open(path)
    json_file = json.load(file)

    mask_lst = []
    name_lst = []
    width = json_file['images'][0]['width']
    height = json_file['images'][0]['height']
    annotations = json_file['annotations']
    n_frame = len(json_file['images'])
    mask = np.zeros((n_frame,width,height), dtype='int')
    for images in json_file['images']:
        name_lst.append(images['file_name'])
    for annot in annotations:
        segmentation = annot['segmentation'][0]
        id = int(annot['id'])
        frame_id = int(annot['image_id'])
        coord_lst = []
        for point in range(0, len(segmentation),2):
            coord_lst.append((round(segmentation[point+1]), round(segmentation[point])))
        tempmask =  polygon2mask((width,height),coord_lst)
        tempmask[tempmask!=0] = id
        mask[frame_id-1] = mask[frame_id-1] + tempmask
    mask_lst.append(mask)
    mask_stack = np.squeeze(np.stack(mask_lst))
    return name_lst, mask_stack

In [232]:
path = '/home/Fabian/ImageData/instances_Series.json'
name_lst, mask_stack = mask_from_json(path)

# Play area

In [61]:
import numpy as np
from tifffile import imwrite
from os import walk
from os.path import join
from concurrent.futures import ThreadPoolExecutor

In [ ]:
folder_path = '/home/Fabian/ImageData/CalciumImmunCell/Macrophage/Laserwound/c1147x1382_003-MaxIP_s2/Masks_Cellpose'

In [68]:
for root, dirs, files in walk(folder_path):
    for file in files:
        if file.endswith('.npy'):            
            npy = np.load(join(root, file), allow_pickle=True).item()
            filename = npy['filename']
            mask = npy['masks']
            imwrite(filename, mask)

UnpicklingError: pickle data was truncated

In [2]:
import pandas as pd
import numpy as np
import torch
from tifffile import imread
from matplotlib.pyplot import imshow

In [23]:
img_path = '/home/Fabian/ImageData/CalciumImmunCell/Macrophage/Amputation/c1147x1382_001-MaxIP_s1/Masks_Cellpose/RFP_s01_f0001_z0001.tif'
mask = imread(img_path)

In [38]:
csv_path = '/home/Fabian/ImageData/TrackingTestFiles/PipelineTest/1382x1166_15%laser@5min-MaxIP-MaxIP_s1/gnn_files/all_data_df.csv'
df = pd.read_csv(csv_path)
pytorch_path = '/home/Fabian/ImageData/TrackingTestFiles/PipelineTest/1382x1166_15%laser@5min-MaxIP-MaxIP_s1/gnn_files/pytorch_geometric_data.pt'
edge_index = torch.load(pytorch_path).edge_index
output_path = '/home/Fabian/ImageData/TrackingTestFiles/PipelineTest/1382x1166_15%laser@5min-MaxIP-MaxIP_s1/gnn_files/raw_output.pt'
output_pred = torch.load(output_path)
outputs = output_pred


In [39]:
decision_threshold = 0.4
frame_ind = 0

In [40]:
nodes_indices = df[df.frame_num==frame_ind].index.values

In [22]:
nodes_indices

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [42]:
np.unique(outputs_hard.bool(), return_counts=True)

(array([False,  True]), array([365123, 794621]))

In [41]:
outputs_soft = torch.sigmoid(outputs)
outputs_hard = (outputs_soft > decision_threshold).int()
connected_indices = edge_index[:, outputs_hard.bool()]

# ind_place = np.argwhere(connected_indices[0, :] == 2)

In [43]:
connected_indices.shape

torch.Size([2, 794621])

In [29]:
edge_index.shape

torch.Size([2, 1159744])

In [44]:
outputs

tensor([34.1735, 28.6891, 28.9476,  ..., 27.8264, 35.9322, 32.3199])

In [46]:
np.unique(outputs_soft)

array([1.0982136e-08, 1.1442487e-08, 1.3596248e-08, ..., 9.9999976e-01,
       9.9999988e-01, 1.0000000e+00], dtype=float32)

In [30]:
outputs.shape

torch.Size([1159744])

In [31]:
outputs_hard.shape

torch.Size([1159744])

In [28]:
connected_indices.shape

torch.Size([2, 794621])

In [27]:
32000 in connected_indices[0,:]

True

In [39]:
labels, count = np.unique(connected_indices[1,:],return_counts=True)
np.argmax(count)

3817

In [9]:
i = 115
ind_place = np.argwhere(connected_indices[0, :] == i)
print(f'{ind_place=}')
next_frame_ind = connected_indices[1, ind_place][0]
print(f'{next_frame_ind=}')

next_node_ind=126

ind_place=tensor([[259]])
next_frame_ind=tensor([126])


In [10]:
condition = connected_indices[1,:] == next_node_ind #delete already assigned nodes from the list to avoid several cells with the same ID per frame
print(f'{condition=}')
# connected_indices = connected_indices[:,~condition] 

condition=tensor([False, False, False,  ..., False, False, False])


In [13]:
connected_indices[:,~condition].shape

torch.Size([2, 20864])

In [11]:
np.unique(condition)

array([False,  True])

In [12]:
connected_indices.shape

torch.Size([2, 20866])

In [18]:
distance = np.array([127.28314892, 116.10340219, 189.21152185, 251.84916121,
        77.0584194 , 461.74126954, 329.67408148, 293.62050337,
       125.67020331, 244.59149617,  60.53924347, 496.00403224,
       616.00324674, 423.0295498 ,  14.56021978, 360.03472055,
       522.06129908, 272.09005862,  39.29376541, 391.15470085,
       294.24479605, 554.23099877,  21.26029163,  28.60069929,
       419.26841045, 329.38882798, 107.68936809, 460.57464107,
       492.44796679,  55.15432893,  81.32035416, 137.29530218,
        24.18677324, 409.0293388 , 258.41439588, 568.64136325,
       365.23417146, 284.80344099,  96.76776323, 643.89828389,
       472.29863434, 302.03476621, 173.76996288,  36.40054945,
       532.08739884, 512.19722764, 609.18634259, 214.57399656,
       458.57714727, 566.34441818, 124.91997438, 293.1705988 ,
       148.76155417, 238.47851056, 348.91402953,  63.03173804,
        49.64876635,  44.01136217,  46.87216658,  82.97590012,
       153.57408636, 218.13069477, 665.53211793, 633.84619586,
        59.03388857, 166.76930173, 485.50695155, 601.63111622,
        74.70609078,  87.00574694, 399.18542058, 291.52872929,
       207.28000386, 440.91835979, 265.66332077, 101.98039027,
       180.70141117, 550.553358  , 363.04407446, 240.35390573,
       664.71121549,  78.77182237, 466.09226554, 134.01492454,
       503.88292291, 101.59724406, 121.69634341, 305.26218239,
       263.1083427 , 621.83679531, 381.76039606, 423.88323864,
       203.03940504, 640.6403047 , 601.23622645, 230.58620947,
       327.48282398,  96.56603958, 657.05098737, 271.18443908,
       116.21101497, 538.18398341, 240.75090862, 105.84894898,
       569.84296082, 259.61702564, 453.84248369, 496.44838604,
       168.66831356, 260.86394921, 157.03502794, 368.43588316,
       290.79889959, 220.91174708, 397.80020111, 331.76648414,
       186.46179233, 236.79949324, 634.07649381, 152.30561382,
       143.60013928, 258.04844506, 194.28072473, 439.00455579,
       185.15128949, 290.29123307, 666.08257746, 315.79265349,
       252.43811123, 136.88316186, 468.00213675, 558.4012894 ,
       267.05991837, 532.8508234 , 392.13518077, 239.45354456,
       613.29356755, 597.64956287, 263.82190963, 581.4894668 ,
       229.13969538, 199.06029237, 505.79936734, 386.00518131,
       355.62058433, 209.60200381, 161.71889191, 220.86421168,
       255.72641631, 432.33436135])

In [22]:
distance_mask = distance < 50

In [27]:
filtered_distance = distance[distance_mask]
min_index = np.argmin(filtered_distance)
nearest_cell = np.where(distance_mask)[0][min_index]
nearest_cell

14

In [25]:
filtered_distance.shape[0]

9

In [80]:
condition = connected_indices[1,:] == 2033

In [83]:
connected_indices = connected_indices[:,~condition]

In [85]:
np.where(connected_indices[1,:]==2033)

(array([], dtype=int64),)

In [93]:
next_node_ind = -2
mitosis = False
if not mitosis and not next_node_ind==-1:
    print('Hello')

Hello


In [86]:
print(np.where(connected_indices[1,:]==2033))
print(connected_indices[:,3817])
print(connected_indices[:,3820])
print(connected_indices[:,3823])

(array([], dtype=int64),)
tensor([2024, 2031])
tensor([2025, 2032])
tensor([2028, 2036])


In [ ]:
np.unique(outputs_soft)

In [100]:
def fill_first_frame(cell_starts):
    cols = ["child_id", "parent_id", "start_frame"]
    df_parent = pd.DataFrame(index=range(len(cell_starts)), columns=cols)
    df_parent.loc[:, ["start_frame", "parent_id"]] = 0
    df_parent.loc[:, "child_id"] = cell_starts
    return df_parent

In [ ]:
def insert_in_specific_col(all_frames_traject, frame_ind, curr_node, next_node):
    if curr_node in all_frames_traject[frame_ind, :]:
        flag = 0
        ind_place = np.argwhere(all_frames_traject[frame_ind, :] == curr_node)
        if frame_ind + 1 < all_frames_traject.shape[0]:
            all_frames_traject[frame_ind + 1, ind_place] = next_node
    else:
        flag = 1
        ind_place = np.argwhere(all_frames_traject[frame_ind, :] == -2)
        while ind_place.size == 0:
            new_col = -2 * np.ones((all_frames_traject.shape[0], 1), dtype=all_frames_traject.dtype)
            all_frames_traject = np.append(all_frames_traject, new_col, axis=1)
            ind_place = np.argwhere(all_frames_traject[frame_ind, :] == -2)
        ind_place = ind_place.min()
        all_frames_traject[frame_ind, ind_place] = curr_node
        if frame_ind + 1 < all_frames_traject.shape[0]:
            all_frames_traject[frame_ind + 1, ind_place] = next_node
    return flag, all_frames_traject

In [205]:
connected_indices = edge_index[:, outputs_hard.bool()]

In [209]:
connected_indices[0, :]

tensor([    0,     1,     2,  ..., 12506, 12507, 12508])

In [205]:
test = np.array(11)

In [206]:
test[0]

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [ ]:
outputs_soft = torch.sigmoid(outputs)
outputs_hard = (outputs_soft > decision_threshold).int()


flag_id0_terminate = False
# extract values from arguments
connected_indices = edge_index[:, outputs_hard.bool()]


# find number of frames for iterations
frame_nums, counts = np.unique(df.frame_num, return_counts=True)
all_frames_traject = np.zeros((frame_nums.shape[0], counts.max())) #crearing matrix with shape (rows=frames, column=max num of label in frame)

# intialize the matrix with -2 meaning empty cell, -1 means end of trajectory,
# other value means the number of node in the graph
all_frames_traject[:, :] = -2
all_trajectory_dict = {}
str_track = ''
df_parents = []

for frame_ind in frame_nums: 
    nodes_indices = df[df.frame_num==frame_ind].index.values # find the places containing frame_ind
    next_frame_indices = np.array([])
    
    if frame_ind == 0:  # for the first frame, we should fill the first row with node indices
        all_frames_traject[frame_ind, :nodes_indices.shape[0]] = nodes_indices
        df_parents.append(fill_first_frame(nodes_indices))
    
    num_starts = 0
    cell_starts = []
    
    for i in nodes_indices:
        if i in connected_indices[0, :]:
            ind_place = np.argwhere(connected_indices[0, :] == i)
            #check how many potential connections one cell has
            if ind_place.shape[-1] > 1: # if more than one connection is possible:
                next_frame_ind = connected_indices[1, ind_place].numpy().squeeze() #get the ID of the potential cells in the next frame
            
                next_frame = df.loc[next_frame_ind, ["centroid_row", "centroid_col"]].values #getting the centroid position for the potential connection points
                curr_node = df.loc[i, ["centroid_row", "centroid_col"]].values #getting the original centroid

                distance = np.sqrt(((next_frame - curr_node) ** 2).sum(axis=-1)) #get the euclidean distance between the node and the possible cells to connect
                nearest_cell = np.argmin(distance, axis=-1) #get the index of the closest cell
                # add to the array
                next_node_ind = next_frame_ind[nearest_cell]

            elif ind_place.shape[-1] == 1:  # one node in the next frame is connected to current node
                next_node_ind = connected_indices[1, ind_place[0]]
            else:  # no node in the next frame is connected to current node -
                # in this case we end the trajectory
                next_node_ind = -1
        else:
            # we dont find the current node in the edge indices matrix - meaning we dont have a connection
            # for the node - in this case we end the trajectory and the cell
            if i == 0:
                flag_id0_terminate = True
            next_node_ind = -1                
        next_frame_indices = np.append(next_frame_indices, next_node_ind) #add the next node index or -1 for track stop into the next_frame_indices list
        # count the number of starting trajectories
        start, all_frames_traject = insert_in_specific_col(all_frames_traject, frame_ind, i, next_node_ind)
        num_starts += start                
        
                
                
                

In [228]:
frame_ind=0
df_parents = []
nodes_indices = df[df.frame_num==frame_ind].index.values # find the places containing frame_ind
next_frame_indices = np.array([])

if frame_ind == 0:  # for the first frame, we should fill the first row with node indices
    all_frames_traject[frame_ind, :nodes_indices.shape[0]] = nodes_indices
    df_parents.append(fill_first_frame(nodes_indices))


num_starts = 0
cell_starts = []

i = nodes_indices[2]


if i in connected_indices[0, :]:
    ind_place = np.argwhere(connected_indices[0, :] == i)
    if ind_place.shape[-1] > 1: # if more than one connection is possible:
        next_frame_ind = connected_indices[1, ind_place].numpy().squeeze()
        next_frame = df.loc[next_frame_ind, ["centroid_row", "centroid_col"]].values
        curr_node = df.loc[i, ["centroid_row", "centroid_col"]].values
        print(f'{curr_node=}')
        distance = np.sqrt(((next_frame - curr_node) ** 2).sum(axis=-1))
        print(f'{distance=}')
        nearest_cell = np.argmin(distance, axis=-1)
        print(f'{nearest_cell=}')
        next_node_ind = next_frame_ind[nearest_cell]
        print(f'{next_node_ind=}')
next_frame_indices = np.append(next_frame_indices, next_node_ind)
print(f'{next_frame_indices=}')
# count the number of starting trajectories
start, all_frames_traject = self.insert_in_specific_col(all_frames_traject, frame_ind, i, next_node_ind)
num_starts += start    


curr_node=array([565., 273.])
distance=array([ 0.        , 66.09841148])
nearest_cell=0
next_node_ind=13
next_frame_indices=array([13.])


In [168]:
def print_value(value, test_list):
    print(value)
    print(test_list)

In [166]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial

In [169]:
unique_stuff = np.unique([90,80,700,50,80])
test_list = 'blbla'
partiaprint = partial(print_value, test_list=test_list)

with ThreadPoolExecutor() as executer:
    executer.map(partiaprint, np.unique([90,80,700,50,80]))


50
blbla
80
blbla
90
blbla
700
blbla


In [108]:
all_frames_traject_path = '/home/Fabian/ImageData/all_frames_traject.csv'
all_frames_traject = np.genfromtxt(all_frames_traject_path, delimiter=',')
cols = ["child_id", "parent_id", "start_frame"]

In [105]:
frame_ind = 2

In [111]:
start1 = np.argwhere(all_frames_traject[frame_ind-1, :] == -1)
start2 = np.argwhere(all_frames_traject[frame_ind-1, :] == -2)

In [116]:
start_node_ids = all_frames_traject[frame_ind, start1].squeeze(axis=1)

In [129]:
cell_starts = start_node_ids[start_node_ids>0]
print(cell_starts)
cell = 23

[23. 24. 25. 27. 29. 30.]


In [134]:
max_dist = 100

In [130]:
finish_node_ids = all_frames_traject[frame_ind - 1, ind_place].squeeze(axis=1)
df_parent = pd.DataFrame(index=range(len(cell_starts)), columns=cols)
df_parent.loc[:, "start_frame"] = frame_ind
finish_cell = df.loc[finish_node_ids, ["centroid_row", "centroid_col"]].values
curr_cell = df.loc[cell, ["centroid_row", "centroid_col"]].values

In [203]:
np.delete(finish_node_ids,[5])

array([16., 18., 14., 15., 19.])

In [ ]:
distance = np.sqrt(((finish_cell - curr_cell) ** 2).sum(axis=-1)) #get the distance from every point
distance_mask = distance < self.max_travel_dist #check that distances are inside the max_travel distance
filtered_distance = distance[distance_mask] #apply the filter on the array
if filtered_distance.size == 0:
    df_parent.loc[ind, "child_id"] = cell
    df_parent.loc[ind, "parent_id"] = 0
    continue
min_index = np.argmin(filtered_distance) #get the smalest distance index in the filtered array
nearest_cell = np.where(distance_mask)[0][min_index] #get back the index from the original array

In [200]:
finish_node_ids

array([16., 18., 14., 15., 19., 20.])

In [132]:
distance = np.sqrt(((finish_cell - curr_cell) ** 2).sum(axis=-1))
distance

array([ 62.20128616, 120.42009799,  64.93843238, 180.62391868,
       185.31055016, 233.85679379])

In [195]:
distance = np.array([28,48,800,12,52,364,75])
max_dist=10

In [ ]:
masked_array = distance < max_dist
filtered = distance[masked_array]
min_index = np.argmin(filtered)
np.where(masked_array)[0][min_index]

In [199]:
filtered.size ==0

True

In [146]:
test_array = np.argmin([17,5,7,9,10,18])

4

In [210]:
all_frames_traject[frame_ind, 0]

21.0

In [261]:
ind_place = np.argwhere(connected_indices[0, :] == 1)
ind_place

tensor([[1]])

In [262]:
next_frame_ind = connected_indices[1, ind_place][0]#.numpy().squeeze()
next_frame_ind

tensor([12])

In [263]:
next_frame = df.loc[next_frame_ind, ["centroid_row", "centroid_col"]].values #getting the centroid position for the potential connection points
curr_node = df.loc[2033, ["centroid_row", "centroid_col"]].values #getting the original centroid
distance = np.sqrt(((next_frame - curr_node) ** 2).sum(axis=-1))
distance

array([358.07541105])

In [265]:
max_dist = 400

In [266]:
masked_array = distance < max_dist
filtered = distance[masked_array]
min_index = np.argmin(filtered)
nearest_cell = np.where(masked_array)[0][min_index]
next_node_ind = int(next_frame_ind[nearest_cell])
next_node_ind

12

In [133]:
max_travel_dist = 100


def find_parent_cell(self, frame_ind, all_frames_traject, df, cell_starts):
    ind_place = np.argwhere(all_frames_traject[frame_ind, :] == -1) #find all indeces were a track ended
    finish_node_ids = all_frames_traject[frame_ind - 1, ind_place].squeeze(axis=1)# find the start IDs in the frame before
    # print(f"frame_ind: {frame_ind}, cell_starts: {cell_starts}, cell_ends: {finish_node_ids}")

    df_parent = pd.DataFrame(index=range(len(cell_starts)), columns=self.cols)
    df_parent.loc[:, "start_frame"] = frame_ind

    if finish_node_ids.shape[0] != 0:
        if self.is_3d:
            finish_cell = df.loc[finish_node_ids, ["centroid_depth", "centroid_row", "centroid_col"]].values
        else:
            finish_cell = df.loc[finish_node_ids, ["centroid_row", "centroid_col"]].values
        for ind, cell in enumerate(cell_starts):
            if self.is_3d:
                curr_cell = df.loc[cell, ["centroid_depth", "centroid_row", "centroid_col"]].values
            else:
                curr_cell = df.loc[cell, ["centroid_row", "centroid_col"]].values

            distance = ((finish_cell - curr_cell) ** 2).sum(axis=-1)
            nearest_cell = np.argmin(distance, axis=-1)
            parent_cell = int(finish_node_ids[nearest_cell])
            df_parent.loc[ind, "child_id"] = cell
            df_parent.loc[ind, "parent_id"] = parent_cell
    else:
        df_parent.loc[:, "child_id"] = cell_starts
        df_parent.loc[:, "parent_id"] = 0

    return df_parent